In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import psycopg2
import glob

from datetime import date


In [2]:
# Function to get current working directory
def current_path():
    print("Current Directory:")
    print(os.getcwd())
    print()
    
# Function to get new directory 
def new_path():
    print("New Directory:")
    print(os.getcwd())
    print()
 
 
# Driver's code
# Printing CWD before
current_path()
 
# Changing the CWD
os.chdir('../../../Work Order - Data/WO_Folders/WO_2022-2023')
 
# Printing CWD after
new_path()

#Itererates and gets the latest file
cwd = os.getcwd()
files = [x for x in os.listdir(cwd) if x.endswith(".csv")]
newest_file = max(files, key = os.path.getctime)
print("Latest CSV Received: ", newest_file)

#Ensuring that the files is chosen:
os.path.exists(newest_file)

Current Directory:
C:\Users\E178162\Documents\Work Orders\DB\Jupyter Notebooks\Data Cleanse

New Directory:
C:\Users\E178162\Documents\Work Orders\Work Order - Data\WO_Folders\WO_2022-2023

Latest CSV Received:  Work_Order_(01_25_2023_02_59_42_PM).csv


True

In [5]:
df = pd.read_csv(newest_file)

df['Date Open'] = pd.to_datetime(df['Created On']).dt.date
df['Date Closed'] = pd.to_datetime(df['Completed']).dt.date #Creates new column and sets data type from DateTime -> Date

df['Date Difference'] = date.today() - df['Date Open'] #Takes the difference between today's date from Date Open
df['Date Difference'] = df['Date Difference'].dt.days #Converts DateTime Type to Integer

df['Duration'] = df['Date Closed'] - df['Date Open'] #Takes the diffrence between Date Closed from Date Open
df['Duration'] = df['Duration'].dt.days #Converts DateTime Type to Integer
df['Duration'] = df['Duration'].abs() #Gets the Absolute Value of Column Values

df['Site_ID'] = df['Source Site'].str[-3:] #Takes the last 3 characters froms string in this case, we get extract the Site Code

df['School Year'] = '2022 - 2023' #Create new column of School Year

df['Duration'] = df['Duration'].fillna(df['Date Difference']) #Fill null values from Duration column with values from Date Difference

df['Row Count'] = 1 #Create new column of Row Count

df = df.drop(columns=['Area/Room #', 'Originator', 'Originator First Name', 'Work requested', 'Cause', 'Action Taken', 'Comments' ,'Longitude', 'Latitude', 'PM Title', 'Completed','Created On', 'Originated']) #Drops/Removes the columns from table

df = df[df['Site_ID'].str.contains("ZZ-|032|941") == False] #Remove any string or cell that contains those characters

df = df.drop_duplicates(subset='Work Order #', keep='first') #Remove any duplicates
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31551 entries, 0 to 31554
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Work Order #     31551 non-null  int64  
 1   WO Status        31551 non-null  object 
 2   Priority         29933 non-null  object 
 3   Work Category    31550 non-null  object 
 4   Problem          31550 non-null  object 
 5   Source Site      31551 non-null  object 
 6   Source Location  30473 non-null  object 
 7   Source User      31052 non-null  object 
 8   Work Type        18293 non-null  object 
 9   Labor Hrs        31551 non-null  float64
 10  Part Cost        31551 non-null  float64
 11  Labor Cost       31551 non-null  float64
 12  Total Hrs        31551 non-null  float64
 13  Grand Total      31551 non-null  float64
 14  Date Open        31551 non-null  object 
 15  Date Closed      27501 non-null  object 
 16  Date Difference  31551 non-null  int64  
 17  Duration    

In [7]:
#Change the file to "updated work orders"
clean_tbl_name = newest_file.lower().replace(" ","").replace("?", "")\
                    .replace("-", "_").replace(r"/","_").replace("\\", "_")\
                    .replace("%", "").replace(")", "").replace(r"(","").replace("$", "")\
                    .replace("pm", "").replace("1", "").replace("2", "").replace("3", "")\
                    .replace("4", "").replace("5", "").replace("6", "").replace("7", "")\
                    .replace("8", "").replace("9", "").replace("0", "").replace("_", "")\
                    .replace("workorder", "updated_work_orders")
clean_tbl_name

'updated_work_orders.csv'

In [8]:
#Modify column names
df.columns = [x.lower().replace(" ","_").replace("?", "").replace("#", "id")\
                    .replace("-", "_").replace(r"/","_").replace("\\", "_")\
                    .replace("%", "").replace(")", "").replace(r"(","").replace("$", "") for x in df.columns]

df.columns

Index(['work_order_id', 'wo_status', 'priority', 'work_category', 'problem',
       'source_site', 'source_location', 'source_user', 'work_type',
       'labor_hrs', 'part_cost', 'labor_cost', 'total_hrs', 'grand_total',
       'date_open', 'date_closed', 'date_difference', 'duration', 'site_id',
       'school_year', 'row_count'],
      dtype='object')

In [9]:
df.dtypes

work_order_id        int64
wo_status           object
priority            object
work_category       object
problem             object
source_site         object
source_location     object
source_user         object
work_type           object
labor_hrs          float64
part_cost          float64
labor_cost         float64
total_hrs          float64
grand_total        float64
date_open           object
date_closed         object
date_difference      int64
duration           float64
site_id             object
school_year         object
row_count            int64
dtype: object

In [10]:
replacements = {
    'object': 'varchar',
    'float64': 'float',
    'int64': 'int',
    'datetime64': 'timestamp',
    'timedelta64[ns]': 'varchar'
}

replacements

{'object': 'varchar',
 'float64': 'float',
 'int64': 'int',
 'datetime64': 'timestamp',
 'timedelta64[ns]': 'varchar'}

In [11]:
col_str = ", ".join("{} {}".format(n, d) for (n,d) in zip(df.columns, df.dtypes.replace(replacements)))
col_str

'work_order_id int, wo_status varchar, priority varchar, work_category varchar, problem varchar, source_site varchar, source_location varchar, source_user varchar, work_type varchar, labor_hrs float, part_cost float, labor_cost float, total_hrs float, grand_total float, date_open varchar, date_closed varchar, date_difference int, duration float, site_id varchar, school_year varchar, row_count int'

In [12]:
#Connect to my PSQL Server
conn_string = "host=localhost\
               dbname='work_orders'\
               user=postgres password='postgres'"

conn = psycopg2.connect(conn_string)
cursor = conn.cursor()
print('opened database successfully!')

opened database successfully!


In [13]:
#drop table
cursor.execute("drop table if exists work_orders;")

In [14]:
#create table
cursor.execute("CREATE TABLE work_orders \
(work_order_id INT NOT NULL PRIMARY KEY, wo_status VARCHAR(50), priority VARCHAR(50), \
work_category VARCHAR(50) REFERENCES work_category(work_category), problem VARCHAR(50), \
source_site VARCHAR(50), source_location VARCHAR(50), source_user VARCHAR(50), work_type VARCHAR(50),\
labor_hrs float, part_cost float, labor_cost float, total_hrs float, grand_total float, \
date_open DATE, date_closed DATE, \
date_difference float, duration float, site_id INT REFERENCES schools(site_id), school_year VARCHAR(25), row_count INT NOT NULL)")

In [15]:
#Change the Directory one more time to move the CSV to new folder

# Function to get current working directory
def current_path():
    print("Current Directory:")
    print(os.getcwd())
    print()
    
# Function to get new directory 
def new_path():
    print("New Directory:")
    print(os.getcwd())
    print()
 
 
# Driver's code
# Printing CWD before
current_path()
 
# Changing the CWD
os.chdir('../../Cleansed_WO_File')
 
# Printing CWD after
new_path()

Current Directory:
C:\Users\E178162\Documents\Work Orders\Work Order - Data\WO_Folders\WO_2022-2023

New Directory:
C:\Users\E178162\Documents\Work Orders\Work Order - Data\Cleansed_WO_File



In [16]:
#Prints CSV to new folder
df.to_csv('updated_work_orders.csv', header=df.columns, index=False, encoding='utf=8')

my_file = open('updated_work_orders.csv')
print('File opened in memory and export successful!')

File opened in memory and export successful!


In [17]:
#upload to db

SQL_STATEMENT = """
COPY work_orders FROM STDIN WITH 
    CSV
    HEADER
    DELIMITER AS ','
"""

cursor.copy_expert(sql = SQL_STATEMENT, file=my_file)
print('file copied to db!')

file copied to db!


In [18]:
cursor.execute("grant select on table work_orders to public")
conn.commit()

cursor.close()
print('CSV imported to DB successfully!')

CSV imported to DB successfully!
